## 第一部分
先就已事先處理的'new_winequality'，進行模型的選擇。
選擇出兩個適合的模型進行下一步採樣比較。

In [ ]:
import pandas as pd
train_df = pd.read_csv('newdataset/new_winequality.csv')

In [1]:
from sklearn.model_selection import train_test_split

# 資料分割 def
def split_data(df, target_column, test_size=0.2, random_state=42):

    X = df.drop(columns=[target_column])
    y = df[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test

### 模型選擇
Logistic Regression
Random Forest
SVM
KNN
Naive Bayes